# Summary
- Data : Bing Corona Virus Search Data (https://github.com/microsoft/BingCoronavirusQuerySet)

- Objectives : Using Bing Search Data relavant to Corona Virus in 2021 from various countries, see the search trends and plot using line/scatter/bubble plots.

> 1) Visualize trends of overall searches/searches by countries using line and scatter plots

> 2) Visualize keywords with most searches on a map.

In [0]:
from datetime import datetime

from geopy.geocoders import Nominatim
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pyspark

In [0]:
def retrieve_column(array, col):
    try:
        return array[col]
    except:
        pass

    
def convert_string_to_timestamp(string):
    try:
        return datetime.strptime(string, '%Y-%M-%d').date()
    except:
        pass


def string_to_int(x):
    try:
        return int(x)
    except:
        return 0

## Connect to GCP Cloud Storage
- In order to allow the Databrick to read from your GCS bucket, you need to create a GCP Service Account with a proper role (ex. Storage Admin) and add to your cluster.
- Go to your project in GCP => IAM & Admin => Service Account => Create Service Account
- Create an account with Storage Admin (or something that gives a storage access) and copy its Principal.

In [0]:
bing_inputs = sc.wholeTextFiles("gs://usf-databricks-gcp/Bing_Searches")

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-3877317448685664>, line 3
      1 from datetime import datetime
----> 3 from geopy.geocoders import Nominatim
      4 import numpy as np
      5 import pandas as pd

ModuleNotFoundError: No module named 'geopy'

In [0]:
bing_inputs.collect()

In [0]:
input = bing_inputs.map(lambda x: x[1]).flatMap(lambda x : x.split("\n"))

In [0]:
parsed_input = input.map(lambda x : x.split("\t"))\
                    .map(lambda x : [convert_string_to_timestamp(retrieve_column(x, 0)),
                                     retrieve_column(x, 1), 
                                     retrieve_column(x, 2),
                                     retrieve_column(x, 3),
                                     retrieve_column(x, 4), 
                                     string_to_int(retrieve_column(x,5))])\
                    .cache()

# Display Daily Count 

Create a line chart showing daily counts

It shows there are patterns in 7 days, where Thursday has the highest value.

In [0]:
daily_count = parsed_input.map(lambda x : [x[0],x[5]])\
                          .reduceByKey(lambda x,y : x+y)\
                          .collect()

daily_count_df = pd.DataFrame(daily_count, columns=['date', 'count'])
fig = px.line(daily_count_df, x='date', y="count")
fig.show()

# Display Daily Count Per Country
For US, France, UK, Australia and Taiwan, create a scatter plot showing daily counts.

Observation : It shows there are patterns in 7 days, where Thursday/Friday has the highest value. UK has the strongest market among all.

In [0]:
country_daily_count = parsed_input.map(lambda x : ((x[4], x[0]), x[5]))\
                                  .reduceByKey(lambda x, y : x+y)\
                                  .map(lambda x : [x[0][0], x[0][1], x[1]]).collect()

country_count_df = pd.DataFrame(country_daily_count, columns=['country', 'date', 'count'])
fig = px.scatter(country_count_df.query('country=="United States" or country== "France" or country=="United Kingdom" or country=="Australia" or country=="Taiwan"'), 
                 x="date", 
                 y="count",
                 color="country")
fig.show()


# Display a map plot 
Create a bubble map showing top 200 (country, word) pairs. 

Observation :  Overall there are many topics relevant to the pandemic, but shows differences depending on its own political/cultural issues such as Lloyds Online Banking in UK, costco online in the US.

In [0]:

geolocator = Nominatim(user_agent="user_name")

def return_with_lon_lat(vals):
    lon_lat = list()
    for val in vals:
        loc = geolocator.geocode(val[0])
        lon_lat.append([loc.longitude, loc.latitude])
    return np.append(np.array(vals), np.array(lon_lat), 1)
    
    
country_word_freq = parsed_input.map(lambda x : ((x[4], x[1]), x[-1]))\
                                .groupByKey()\
                                .mapValues(lambda x : sum(x))\
                                .map(lambda val : [val[0][0], val[0][1], val[1]])\
                                .sortBy(lambda x : x[2], ascending = False)\
                                .take(200)


In [0]:
limits = [(0,50),(50,100),(100,150),(150,200)]
colors = ["royalblue","crimson","lightseagreen","orange","lightgrey"]

fig = go.Figure(go.Scattergeo())
fig.update_layout(height=1000)

country_word_freq_df = pd.DataFrame(return_with_lon_lat(country_word_freq), columns=['country', 'keyword', 'count', 'lon', 'lat'])

for i in range(len(limits)):
    lim = limits[i]
    df_sub = country_word_freq_df[lim[0]:lim[1]]
    fig.add_trace(go.Scattergeo(lon = df_sub['lon'],
                                lat = df_sub['lat'],
                                text = df_sub['keyword'] + '<br>' + df_sub['country'] + '<br>' + df_sub['count'],
                                marker = dict(
                                    color = colors[i],
                                    size = df_sub['count'].astype('int32')/10,
                                    line_width=0.5,
                                    sizemode = 'area'
                            ),
                             name = 'Top {0} - {1}'.format(lim[0],lim[1])))


In [0]:
fig.show()